<a href="https://colab.research.google.com/github/pinnouse/zeroshot-unsurpervised-mt/blob/main/CSC413_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache_beam mwparserfromhell
!pip install transformers
!pip install datasets
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

# multilingual CLIP pretrained
# https://github.com/FreddeFrallan/Multilingual-CLIP
!pip install multilingual-clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not cur

# Data Processing

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.simple")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
text = map(lambda x: x['text'].replace("\n", ' ').split(". "), dataset['train'])
for t in text:
  print(t)
  break

['April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May', 'It is one of four months to have 30 days', ' April always begins on the same day of week as July, and additionally, January in leap years', 'April always ends on the same day of the week as December', " April's flowers are the Sweet Pea and Daisy", 'Its birthstone is the diamond', 'The meaning of the diamond is innocence', ' The Month   April comes between March and May, making it the fourth month of the year', 'It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year', ' April begins on the same day of the week as July every year and on the same day of the week as January in leap years', "April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart", ' In common years, April starts on the same day of the week as October of the previous ye

In [ ]:
import torch
import torchtext
import clip
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

glove = torchtext.vocab.GloVe(name="6B", dim=50)
ft = torchtext.vocab.FastText(language="simple")

In [ ]:
train_data = []

for (i, t) in enumerate(text):
  if i > 100:
    break
  sentence = t[0][:64]
  tokenized = clip.tokenize([sentence]).to(device)
  text_embed = model.encode_text(tokenized)[0]
  # glove_embed = np.array([glove[w] for w in sentence.lower().split(' ')])
  # ft_embed = np.array([ft[w] for w in sentence.lower().split(' ')])
  train_data.append((text_embed, glove_embed, ft_embed, sentence))

<ipython-input-88-aaef1c69e440>:9: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  glove_embed = np.array([glove[w] for w in sentence.lower().split(' ')])
<ipython-input-88-aaef1c69e440>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  glove_embed = np.array([glove[w] for w in sentence.lower().split(' ')])
<ipython-input-88-aaef1c69e440>:10: FutureWarning: The input o

In [ ]:
from torch import nn

class Model(nn.Module):
  def __init__(self, dim_size=512, vocab_size=100):
    super().__init__()

    # decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
    # self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
    self.linear = nn.Linear(dim_size, 200)
    self.out = nn.Linear(200, vocab_size)

  # def forward(self, tgt, embed):
  def forward(self, embed):
    # x = self.decoder(tgt, embed)
    x = self.linear(embed)
    out = self.out(x)
    return out

# tgt = torch.rand(3, 512)
# emb = text_embed.unsqueeze(0).to('cpu').float()
m = Model(50).to(device)
emb = glove_embed[0].to(device)
m(emb)

tensor([ 0.0428, -0.0741,  0.0274, -0.1030,  0.0718, -0.0057, -0.0272, -0.0181,
        -0.0273, -0.0299, -0.0094, -0.0501,  0.1002,  0.0049, -0.0288,  0.0943,
        -0.0523,  0.0017, -0.0465,  0.0577, -0.0921,  0.0516, -0.1224, -0.0045,
         0.0420,  0.0309, -0.0411,  0.0636,  0.0137, -0.0598,  0.0636,  0.0916,
         0.0505,  0.0959,  0.0285,  0.0749, -0.0593, -0.1139, -0.0999, -0.0619,
        -0.0224, -0.0639,  0.0283,  0.1198,  0.0741,  0.0672,  0.0334, -0.0489,
        -0.1361, -0.0407, -0.0183,  0.0418, -0.0190,  0.0086,  0.0899, -0.0889,
         0.0543, -0.0384, -0.1100,  0.0264,  0.0350,  0.1541, -0.0813,  0.0545,
         0.0974, -0.0133,  0.0315,  0.0020,  0.0708,  0.0498,  0.0580,  0.0529,
        -0.0125,  0.0076, -0.0039, -0.0302, -0.1122,  0.0013, -0.0889,  0.0372,
        -0.0076,  0.1071, -0.0120,  0.0055,  0.0826,  0.0725,  0.0828, -0.0580,
        -0.0491, -0.0109,  0.1005,  0.0774,  0.0080, -0.0033, -0.0349, -0.0132,
         0.0623, -0.0033,  0.0979, -0.01

# GAN+Autoencoder Idea
![Diagram](https://media.discordapp.net/attachments/1085342382679138385/1090810686793318491/image.png?width=1440&height=592)
Orange represents the zeroshot translation task

## Define Models

We will create 4 models:
1. Monolingual generator (expert at i.e. FR, DE) - transformer encoder+decoder
2. CLIP Embedding Decoder - transformer decoders
3. Discriminator - linear neural network (subject to change)
4. Translator - two linear parts from embedding space to embedding space (thought of as a transform function from FR -> EN, and the inverse EN -> FR)

We refer to 2 agents that are learning to understand their respective languages, both represented by transformers.
`Base expert` is the agent that 'understands' English (from CLIP).
`Other expert` is agent that is an expert of the other language.

In [ ]:
# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x: Tensor) -> Tensor:
    """
    Args:
        x: Tensor, shape [seq_len, batch_size, embedding_dim]
    """
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

# https://jamesmccaffrey.wordpress.com/2022/09/09/simplest-transformer-seq-to-seq-example/
class Generator(nn.Module): # same language/monolingual
  def __init__(self, ntoken: int, d_model: int = 512, nhead: int = 4, d_hid: int = 512,
               nlayers: int = 4, dropout: float = 0.5):
    super().__init__()

    self.model_type = 'Transformer'

    self.embed = nn.Embedding(ntoken, d_model)

    self.pos_encoder = PositionalEncoding(d_model, dropout)
    encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    self.encoder = nn.Embedding(ntoken, d_model)

    decoder_layers = TransformerDecoderLayer(d_model, nhead, d_hid, dropout)
    self.decoder = nn.TransformerDecoder(d_model)

    self.d_model = d_model

    self.dense = nn.linear(d_model, ntoken)

    self.init_weights()

  def init_weights(self) -> None:
    initrange = 0.1
    self.encoder.weight.data.uniform_(-initrange, initrange)
    self.decoder.weight.data.uniform_(-initrange, initrange)
    self.dense.bias.data.zero_()

  def forward(self, src, tgt, tgt_mask):
    s = self.embed(src)
    t = self.embed(tgt)

    s = self.pos_enc(s)
    t = self.pos_enc(t)

    e = self.encoder(s)
    d = self.decoder(tgt, e, tgt_mask=tgt_mask)
    z = nn.functional.softmax(self.dense(d))
    return z

class Translator(nn.Module):
  def __init__(self, i_embed_size = 512, o_embed_size = 512, nlayers = 1, hidden = 1024):
    super().__init__()
    
    # based off https://arxiv.org/abs/1809.03633
    self.G = nn.Sequential(nn.linear(i_embed_size, hidden))
    self.F = nn.Sequential(nn.linear(o_embed_size, hidden))
    for i in range(nlayers-1):
      self.G.append(nn.linear(hidden, hidden))
      self.F.append(nn.linear(hidden, hidden))
    self.G.append(nn.linear(hidden, o_embed_size))
    self.F.append(nn.linear(hidden, i_embed_size))

  def forward(self, i_emb):
    o = G(i_emb)
    i = F(o) # inverse, ideally returns i_emb
    return o, i

## Train Loop

PyTorch training for GAN:
[PyTorch blog](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

In [ ]:
from torch import optim
# https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.Adafactor
from transformers.optimization import Adafactor

generator = Generator(10000) # temp vocab size

context_length = 64

# TODO: add device specific code

def train(real_decoder, generator, discriminator, translate, # our four models
          real_train, other_train, real_valid = None, other_valid = None,
          epochs = 10, batch_size = 256):
  batch_data = []

  criterion = nn.CrossEntropyLoss()

  # r_optim = optim.RAdam(real_decoder.parameters())
  # g_optim = optim.RAdam(generator.parameters())
  r_optim = Adafactor(real_decoder.parameters())
  g_optim = Adafactor(generator.parameters())
  t_optim = Adafactor(translate.parameters())
  d_optim = optim.Adam(discriminator.parameters())

  r_iterations = real_train.shape[0] // batch_size
  o_iterations = other_train.shape[0] // batch_size

  n = min(r_iterations, o_iterations)

  for e in epochs:
    epoch_loss = 0
    print(f'Epoch {e+1}:')
    for i, (r_x, o_x) in enumerate(data_loader):
      if r_x.shape[0] % batch_size != 0: # not full batch
        break
      if (i + 1) % 100 == 0:
        print(f'Iteration {i+1} of {n}')

      # learn decoder
      r_optim.zero_grad()
      ...

      # "other" generator autoencoding
      # https://jamesmccaffrey.wordpress.com/2022/09/09/simplest-transformer-seq-to-seq-example/
      tgt_in = tgt[:,:-1]
      tgt_expect = tgt[:,1:]
      t_mask = T.nn.Transformer.generate_square_subsequent_mask(context_length)

      output = generator(src, tgt_in, tgt_mask=t_mask) # [bs,seq,vocab]

      # get preds shape to conform to tgt_expect
      output = output.permute(0,2,1)  # now [bs, vocab, seq]

      loss = criterion(output, tgt_expect)
      epoch_loss += loss.item()

      g_optim.zero_grad()
      loss.backward()
      g_optim.step()